# Pytorch Tutorial

Pytorch is a popular deep learning framework and it's easy to get started.

In [1]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torch.nn.functional as F
import torch.optim as optim
import argparse
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import time

BATCH_SIZE = 128
NUM_EPOCHS = 5

First, we read the mnist data, preprocess them and encapsulate them into dataloader form.

In [2]:
# preprocessing
normalize = transforms.Normalize(mean=[.5], std=[.5])
transform = transforms.Compose([transforms.ToTensor(), normalize])

# download and load the data
train_dataset = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./mnist/', train=False, transform=transform, download=False)

# encapsulate them into dataloader form
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

Then, we define the model, object function and optimizer that we use to classify.

In [5]:
class SimpleNet(nn.Module):
    # TODO:define model
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)
    

model = SimpleNet()

# TODO:define loss function and optimiter
criterion = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

Next, we can start to train and evaluate!

In [8]:
# train and evaluate
model.train()
for epoch in range(NUM_EPOCHS):
    for images, labels in tqdm(train_loader):
        # TODO:forward + backward + optimize
        images = torch.autograd.Variable(images)
        labels = torch.autograd.Variable(labels)
        
        optimizer.zero_grad()
        
        output_lables = model(images)
        loss = criterion(output_lables, labels)
        
        loss.backward()
        optimizer.step()
        
        
    # evaluate
    # TODO:calculate the accuracy using traning and testing dataset
model.eval()
total_train = 0
correct_train = 0
with torch.no_grad():
    for data in train_loader:
        images, labels = data
        output_lables = model(images)
        predict = output_lables.max(1, keepdim=True)[1]  # get the index of the max log-probability
        correct_train += predict.eq(labels.view_as(predict)).sum().item()
        total_train += labels.size(0)

train_accuracy = correct_train / total_train

total_test = 0
correct_test = 0
for data in train_loader:
    images, labels = data
    output_lables = model(images)
    predict = output_lables.max(1, keepdim=True)[1]  # get the index of the max log-probability
    correct_test += predict.eq(labels.view_as(predict)).sum().item()
    total_test += labels.size(0)
    
test_accuracy = correct_test / total_test
print('Training accuracy: %0.2f%%' % (train_accuracy*100))
print('Testing accuracy: %0.2f%%' % (test_accuracy*100))

  0%|          | 0/468 [00:00<?, ?it/s]

  0%|          | 2/468 [00:00<00:38, 12.08it/s]

  1%|          | 4/468 [00:00<00:46, 10.02it/s]

  1%|          | 5/468 [00:00<00:49,  9.43it/s]

  1%|▏         | 6/468 [00:00<01:14,  6.16it/s]

...

 99%|█████████▉| 464/468 [00:28<00:00, 16.14it/s]

100%|█████████▉| 466/468 [00:29<00:00, 16.11it/s]

100%|██████████| 468/468 [00:29<00:00, 16.13it/s]

Training accuracy: 96.84%

Testing accuracy: 96.83%

#### Q5:
Please print the training and testing accuracy.